In [3]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [4]:
import json
 
# Opening JSON file
f = open('./CubaCrawler/cubadebate.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
f.close()
social_net = 'cubadebate'
text = []
for obj in data:
    text.append(obj['text'])
    for c in obj['comments']:
        text.append(c['text'])

In [5]:
import progressbar
def get_progressbar(N, name = ""):
    return progressbar.ProgressBar(
        maxval=N, 
        widgets=[progressbar.Bar('#', '[', ']'), 
        name, 
        progressbar.Percentage()])

In [6]:
import spacy
nlp = spacy.load("es_core_news_sm")

words = []
bar = get_progressbar(len(text), ' tokenizer text ')
bar.start()
for i, t in enumerate(text):
  doc = nlp(t)
  for token in doc:
    words.append(token.text)
  bar.update(i+1)
bar.finish()

/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[#########################################################] tokenizer text 100%


In [7]:
chars = sorted(list(set(words)))
char_to_num = dict((c, i) for i , c in enumerate(chars))

input_len = len(words)
vocab_len = len(chars)

print(vocab_len, input_len)

52293 802451


In [8]:
seq_len = 100
x, y  = [], []

In [9]:
for i in range(0, input_len - seq_len):
    in_seq = words[i: i+ seq_len]
    out_seq = words[i+ seq_len]
    x.append([char_to_num[char] for char in in_seq])
    y.append(char_to_num[out_seq])

n_pattern = len(x)

In [10]:
X = numpy.reshape(x, (n_pattern, seq_len, 1))
X = X/float(vocab_len)
y = np_utils.to_categorical(y)

In [11]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X, y, 32)

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape = (X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax')
)

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-24 20:32:11.177577: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-24 20:32:11.178301: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 128)          66560     
                                                                 
 dropout (Dropout)           (None, 100, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 64)           49408     
                                                                 
 dropout_1 (Dropout)         (None, 100, 64)           0         
                                                                 
 dense (Dense)               (None, 100, 52293)        3399045   
                                                                 
Total params: 3,515,013
Trainable params: 3,515,013
Non-trainable params: 0
_________________________________________________________________


In [13]:
import os
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [14]:
model.fit(train_gen, epochs=4, batch_size=128, callbacks=[cp_callback])

Epoch 1/4


2022-05-24 20:32:11.829897: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/keras/engine/trainin

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
start = numpy.random.randint(0, len(x) - 1)
pattern = x[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

In [ ]:
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]